Importing images from the image folder and Creating a dataframe with image names and image path in columns

In [6]:
import os 
import pandas as pd 
# Defining image folder path 
image_folder = 'C:/Users/PC/Desktop/Bala/Datascientest/Rakuton/project/Image model/Image processing and cropping/image_train' 

image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')] 

# Creating a dataframe with image names and paths 
df_image = pd.DataFrame({ 'image_name': image_files, 'image_path': [os.path.join(image_folder, f) for f in image_files] }) 

df_image.head()

,image_name,image_path
0,image_1000076039_product_580161.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...
1,image_1000089455_product_348990858.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...
2,image_1000092894_product_353108104.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...
3,image_1000093804_product_343306951.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...
4,image_1000095646_product_344209267.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...


In [7]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84916 entries, 0 to 84915
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  84916 non-null  object
 1   image_path  84916 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


Extracting product ID and image ID from image names

In [8]:
df_image['productid'] = df_image['image_name'].str.split('ct_').str[-1]
df_image['productid'] = df_image['productid'].str.split('.jpg').str[0]

df_image['imageid'] = df_image['image_name'].str.split('image_').str[-1]
df_image['imageid'] = df_image['imageid'].str.split('_p').str[0]

df_image.head()

,image_name,image_path,productid,imageid
0,image_1000076039_product_580161.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,580161,1000076039
1,image_1000089455_product_348990858.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,348990858,1000089455
2,image_1000092894_product_353108104.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,353108104,1000092894
3,image_1000093804_product_343306951.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,343306951,1000093804
4,image_1000095646_product_344209267.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,344209267,1000095646


Spliting dataframe into 4 parts

In [9]:
#since the file is huge, we split the data into 4 parts and get information on image dimension, pixel, non-image from individual files and merge it later
import numpy as np
# Split columns into 4 parts
row_splits = np.array_split(df_image, 4)  # axis=1 for rowwise split

# Access column splits like this:
df1, df2, df3, df4 = row_splits

c:\Users\PC\Desktop\Bala\Datascientest\Rakuton\project\venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Extracting image dimensions into a column 'image_dimension'

In [10]:
from PIL import Image
import numpy as np

def get_image_dimensions(image_path):
     try: 
         with Image.open(image_path) as img:
             return img.size          # Returns (width, height) 
     except Exception as e: 
         print(f"Error opening image {image_path}: {e}") 
         return None          # Return None if there's an error      

Extracting pixel from images into a column 'image_pixel'

In [11]:
from PIL import Image
import numpy as np

def get_image_pixels(image_path):
     try: 
         with Image.open(image_path) as img:
             img_array = np.array(img)
             return img_array          # Returns (width, height) 
     except Exception as e: 
         print(f"Error opening image {image_path}: {e}") 
         return None          # Return None if there's an error

Extracting text from the images

In [12]:
import cv2
def preprocess_image(image_path):
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        return None
    
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply thresholding to get a binary image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    return thresh



def extract_text_from_image(image_path):
    processed_image = preprocess_image(image_path)
    if processed_image is None:
        return "Image not found"

    text = pytesseract.image_to_string(processed_image)

    return text.strip()


Finding blank image files

In [13]:
import cv2
import numpy as np

def is_uniform_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        return True  # Image not found 

    # Get the shape of the image
    height, width, channels = img.shape

    # Extract the first pixel value
    first_pixel_value = img[0, 0]

    # Check if all pixels have the same value as the first pixel
    # Using NumPy's array operations to compare all pixel values
    if np.all(img == first_pixel_value):
        return True  # The image is uniform (all pixels are the same i.e., it is solid color image)
    
    return False  # The image is not uniform (Image is present)

Measuring sharpness of product images (calculating Laplacian variance)

In [14]:
import cv2
import numpy as np

def calculate_sharpness(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image is loaded correctly
    if image is None:
        print("Error: Could not open or find the image.")
        return None
    
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply the Laplacian filter
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
    
    # Calculate the variance
    variance = laplacian.var()
    
    return variance



 #Create a new column for image_dimensions, image_pixel, extracted_text and image_is_blank for df1 and saved as df1_image_process.csv

In [15]:
df1['image_dimension'] = df1['image_path'].apply(get_image_dimensions)

In [16]:
df1['image_pixel'] = df1['image_path'].apply(get_image_pixels)

KeyboardInterrupt: 

In [ ]:
df1["image_is_blank"] = df1["image_path"].apply(is_uniform_image)

In [ ]:
df1['extracted_text']= df1['image_path'].apply(extract_text_from_image)

In [ ]:
df1['sharpness_score']= df1['image_path'].apply(calculate_sharpness)

In [ ]:
df1.to_csv("df1_image_processed.csv")

#Create a new column for image_dimensions, image_pixel and image_is_blank for df2 and saved as df2_image_process.csv

In [ ]:
df2['image_dimension'] = df2['image_path'].apply(get_image_dimensions)

In [ ]:
df2['image_pixel'] = df2['image_path'].apply(get_image_pixels)

In [ ]:
df2["image_is_blank"] = df2["image_path"].apply(is_uniform_image)

In [ ]:
df2['extracted_text']= df2['image_path'].apply(extract_text_from_image)


In [ ]:
df2['sharpness_score']= df2['image_path'].apply(calculate_sharpness)

In [ ]:
display(df2.info())
display(df2['image_is_blank'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21229 entries, 21229 to 42457
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   image_name       21229 non-null  object
 1   image_path       21229 non-null  object
 2   productid        21229 non-null  object
 3   imageid          21229 non-null  object
 4   image_dimension  21229 non-null  object
 5   image_pixel      21229 non-null  object
 6   image_is_blank   21229 non-null  bool  
dtypes: bool(1), object(6)
memory usage: 1016.0+ KB


None

array([False])

In [ ]:
df2.to_csv("df2_image_processed.csv")

#Create a new column image_dimensions, image_pixel and image_is_blank for df3 and saved as df3_image_process.csv

In [ ]:
df3['image_dimension'] = df3['image_path'].apply(get_image_dimensions)

In [ ]:
df3['image_pixel'] = df3['image_path'].apply(get_image_pixels)

In [ ]:
df3["image_is_blank"] = df3["image_path"].apply(is_uniform_image)

In [ ]:
df3['extracted_text']= df3['image_path'].apply(extract_text_from_image)


In [ ]:
df3['sharpness_score']= df3['image_path'].apply(calculate_sharpness)

In [ ]:
display(df3.info())
display(df3['image_is_blank'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21229 entries, 42458 to 63686
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   image_name       21229 non-null  object
 1   image_path       21229 non-null  object
 2   productid        21229 non-null  object
 3   imageid          21229 non-null  object
 4   image_dimension  21229 non-null  object
 5   image_pixel      21229 non-null  object
 6   image_is_blank   21229 non-null  bool  
dtypes: bool(1), object(6)
memory usage: 1016.0+ KB


None

array([False,  True])

In [ ]:
df3.to_csv("df3_image_processed.csv")

#Create a new column image_dimensions, image_pixel and image_is_blank for df4 and saved as df4_image_process.csv

In [ ]:
df4['image_dimension'] = df4['image_path'].apply(get_image_dimensions)

In [ ]:
df4['image_pixel'] = df4['image_path'].apply(get_image_pixels)

In [ ]:
df4["image_is_blank"] = df4["image_path"].apply(is_uniform_image)

In [ ]:
df4['extracted_text']= df4['image_path'].apply(extract_text_from_image)


In [ ]:
df4['sharpness_score']= df4['image_path'].apply(calculate_sharpness)

In [ ]:
display(df4.info())
display(df4['image_is_blank'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21229 entries, 63687 to 84915
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   image_name       21229 non-null  object
 1   image_path       21229 non-null  object
 2   productid        21229 non-null  object
 3   imageid          21229 non-null  object
 4   image_dimension  21229 non-null  object
 5   image_pixel      21229 non-null  object
 6   image_is_blank   21229 non-null  bool  
dtypes: bool(1), object(6)
memory usage: 1016.0+ KB


None

array([False])

In [ ]:
df4.to_csv("df4_image_processed.csv")

Extracting text from the images

Opening the CSV file containing image informations

In [ ]:
import pandas as pd
df1 = pd.read_csv("df1_image_processed.csv")
df2 = pd.read_csv("df2_image_processed.csv")
df3 = pd.read_csv("df3_image_processed.csv")
df4 = pd.read_csv("df4_image_processed.csv")

Checking for number of blank and solid color images

In [ ]:
#count bool counting, counts the number of True or 1
display(df1['image_is_blank'].sum())
display(df2['image_is_blank'].sum())
display(df3['image_is_blank'].sum())
display(df4['image_is_blank'].sum())

np.int64(2)

np.int64(0)

np.int64(5)

np.int64(0)

In [ ]:
df_image = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84916 entries, 0 to 84915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       84916 non-null  int64 
 1   image_name       84916 non-null  object
 2   image_path       84916 non-null  object
 3   productid        84916 non-null  int64 
 4   imageid          84916 non-null  int64 
 5   image_dimension  84916 non-null  object
 6   image_pixel      84916 non-null  object
 7   image_is_blank   84916 non-null  bool  
dtypes: bool(1), int64(3), object(4)
memory usage: 4.6+ MB


Categorizing sharpness score

In [ ]:
def categorize_sharpness(sharpness_score):
    if sharpness_score >= 200:
        return 'Sharp'
    elif 100 >= sharpness_score < 200:
        return 'Moderately sharp'
    else:
        return 'Blurry'
    
df_image['image_quality'] = df_image['sharpness_score'].apply(categorize_sharpness)

In [ ]:
df_noimage = df_image.loc[df_image['image_is_blank'] == True]
df_noimage

,Unnamed: 0,image_name,image_path,productid,imageid,image_dimension,image_pixel,image_is_blank
17384,17384,image_1137819811_product_1892606336.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,1892606336,1137819811,"(500, 500)",[[[255 255 255]\n [255 255 255]\n [255 255 2...,True
17865,17865,image_1142089742_product_884747735.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,884747735,1142089742,"(500, 500)",[[[255 255 255]\n [255 255 255]\n [255 255 2...,True
43852,43852,image_1263803373_product_3921649686.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,3921649686,1263803373,"(500, 500)",[[[176 215 136]\n [176 215 136]\n [176 215 1...,True
43853,43853,image_1263803704_product_3921650414.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,3921650414,1263803704,"(500, 500)",[[[191 162 62]\n [191 162 62]\n [191 162 ...,True
43854,43854,image_1263803707_product_3921650416.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,3921650416,1263803707,"(500, 500)",[[[191 162 62]\n [191 162 62]\n [191 162 ...,True
47465,47465,image_1271791205_product_3894592691.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,3894592691,1271791205,"(500, 500)",[[[255 255 255]\n [255 255 255]\n [255 255 2...,True
59884,59884,image_1302249863_product_3793782107.jpg,C:/Users/PC/Desktop/Bala/Datascientest/Rakuton...,3793782107,1302249863,"(500, 500)",[[[255 255 255]\n [255 255 255]\n [255 255 2...,True


In [ ]:
df_image.to_csv("df_image_20241024.csv")